In [5]:
# [+] PySpark 임포트 및 설정
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('key-value-rdd-operations')
sc = SparkContext(conf=conf)

In [7]:
"""
    groupBy(f): 입력 함수를 기준으로 그룹핑
    - [+] 리스트로부터 rdd 생성
    - 리스트: [1, 1, 2, 3, 5, 8]
""" 

rdd = sc.parallelize([1, 1, 2, 3, 5, 8])

In [8]:
# [+] groupBy() 를 이용한 홀수/짝수 그룹핑

groups = rdd.groupBy(lambda x: x % 2).collect()

# 주의! .collect()안하고 for문 돌릴 수 없음. collect() 해서 파이썬 객체로 만들어야함

for k, v in groups:
    print(k, list(v))

1 [1, 1, 3, 5]
0 [2, 8]


In [9]:
"""
    groupByKey(): Key 값을 기준으로 그룹핑
    - [+] 리스트로부터 key-value rdd 생성
    - 리스트: [('a', 1), ('b', 1), ('a', 1)]
"""
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)])
rdd.collect()

[('a', 1), ('b', 1), ('a', 1)]

In [10]:
# [+] groupByKey() 를 이용한 그룹핑
groups = rdd.groupByKey()
groups.collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x1e6ed61a700>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x1e6ed62f070>)]

In [11]:
for k, v in groups.collect():
    print(k, list(v))

a [1, 1]
b [1]


In [12]:
# len(): Python 객체의 길이를 리턴
value = [1, 1]
len(value)

2

In [13]:
# 그룹 별 원소 개수 계산
counts = groups.mapValues(len)
counts.collect()

[('a', 2), ('b', 1)]

In [19]:
"""
    [+] groupBy() 예제
    - 리스트로부터 RDD 객체 생성
    - 리스트: ['C', 'C++', 'Python', 'Java', 'C#']
    - 원소의 첫 번째 문자 값을 기준으로 그룹핑
    - 기대 결과:
        J ['Java']
        C ['C', 'C++', 'C#']
        P ['Python']
    - 그룹핑 결과를 리스트 객체로 출력
"""

groups = sc.parallelize(['C', 'C++', 'Python', 'Java', 'C#']).groupBy(lambda x: x[0]).collect()

In [20]:
for k, v in groups:
    print(k, list(v))

C ['C', 'C++', 'C#']
P ['Python']
J ['Java']


In [21]:
"""
    groupByKey() 예제
"""

# list 객체로부터 3개의 파티션을 갖는 Key-Value RDD 생성하기
rdd = sc.parallelize([  # 과목 별 점수 데이터
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [22]:
# getNumPartitions(): RDD 객체의 파티션 수 확인하는 Actions
rdd.getNumPartitions()

3

In [29]:
# [+] Key 별로 그룹핑 및 결과 확인
groups = rdd.groupByKey()
groups.collect()

[('Science', <pyspark.resultiterable.ResultIterable at 0x1e6ed62f7f0>),
 ('Math', <pyspark.resultiterable.ResultIterable at 0x1e6ed6b2160>),
 ('English', <pyspark.resultiterable.ResultIterable at 0x1e6ed6b26a0>)]

In [30]:
# [+] for 문을 통한 그룹핑 결과 출력
for k, v in groups.collect():
    print(k, list(v))

Science [7, 6, 9, 5]
Math [7, 2, 8, 3]
English [7, 4, 9, 4]


In [32]:
# 2개의 파티션으로 나누어 키 별로 그룹핑 및 결과 확인 -> 중간 연산 시에도 파티션 수 바꿀 수 있음.
rdd.groupByKey(2).getNumPartitions()

2

In [33]:
"""
    reduce() vs reduceByKey()

"""

# [+] reduce() 를 이용한 총합 구하기
sc.parallelize([1, 2, 3, 4, 5]).reduce(lambda x, y: x + y)

15

In [34]:
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)])

# [+] reduceByKey() 를 이용한 Key 별 총합 구하기
rdd.reduceByKey(lambda x, y: x + y).collect()

[('a', 2), ('b', 1)]

In [35]:
# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [36]:
# reduceByKey() 를 이용한 과목 별 점수 총합 구하기
rdd.reduceByKey(lambda x, y: x + y).collect()

[('Science', 27), ('Math', 20), ('English', 24)]

In [37]:
"""
    mapValues() vs map()
"""
# mapValues(): value 에만 함수를 적용

rdd = sc.parallelize([
    ('a', ['apple', 'banana', 'lemon']),
    ('b', ['grapes'])
])

rdd.mapValues(len).collect()

[('a', 3), ('b', 1)]

In [38]:
# map(): key, value 에 함수를 적용
rdd.map(len).collect()  # 결과가 mapValues()와 다른 이유는?

[2, 2]

In [39]:
"""
    countByKey(): Key 별 원소 개수를 계산
"""

rdd = sc.parallelize([
    ('a', 1), ('b', 1), ('a', 1)
])

rdd.countByKey()

defaultdict(int, {'a': 2, 'b': 1})

In [40]:
"""
    keys(): key 값으로 구성된 RDD 생성(transformation)
"""

# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

rdd.keys().collect()

['Math',
 'Math',
 'English',
 'Science',
 'English',
 'English',
 'Math',
 'Math',
 'English',
 'Science',
 'Science',
 'Science']

In [44]:
# [+] 유니크한 키 개수 출력
rdd.keys().distinct().count()

3

In [43]:
# [+] 유니크한 키 값 출력
rdd.keys().distinct().collect()

['Science', 'Math', 'English']

In [46]:
"""
    Join 연산
    - join(): 내부조인
    - leftOuterJoin(): 외부조인(첫 번째 RDD 중심)
    - rightOuterJoin(): 외부조인(두 번째 RDD 중심)
"""

rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2)])

In [47]:
rdd1.join(rdd2).collect() # join(): Inner Join(두 RDD의 교집합)

[('a', (1, 2))]

In [48]:
rdd1.leftOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (1, 2))]

In [49]:
rdd1.rightOuterJoin(rdd2).collect()

[('a', (1, 2))]